In [1]:
import torch
from torch_geometric.data import Data, Dataset, DataLoader, InMemoryDataset
import numpy as np
from torch.nn.functional import cosine_similarity

In [2]:
def read_edge(file_path):
    edges = []
    with open(file_path) as file:
        for line in file:
            edge = line.strip().split()  # 根据文件格式解析边的信息
            edges.append((int(edge[0]), int(edge[1])))  # 将边的起始节点和目标节点添加到列表中
        for a in range(1,348):
            edges.append([0,a])
        return np.array(edges)
edges=read_edge(r'facebook\0.edges')
edges

array([[236, 186],
       [122, 285],
       [ 24, 346],
       ...,
       [  0, 345],
       [  0, 346],
       [  0, 347]])

In [3]:
# 從文件中讀取節點特徵向量
def read_node_features(ego_file_path,file_path):
    node_features = []
    with open(ego_file_path) as file:
        lines = file.readlines()
        for line in lines:
            # 根據文件格式擷取節點特徵向量
            features = line.strip().split()
            features = [float(f) for f in features]
            node_features.append(features)
    with open(file_path) as file:
        lines = file.readlines()
        a=1
        for line in lines:
            # 根據文件格式擷取節點特徵向量
            features = line.strip(str(a)).split()
            features = [float(f) for f in features]
            node_features.append(features)
            a=a+1
        return np.array(node_features)

# 讀取節點特徵向量數據
node_features = read_node_features(r'facebook\0.egofeat',r'facebook\0.feat')

In [4]:
x = torch.tensor(node_features, dtype=torch.float)  # node features
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()  # edge關係
data = Data(x=x, edge_index=edge_index)
print(data.edge_index)
data.x

tensor([[236, 122,  24,  ...,   0,   0,   0],
        [186, 285, 346,  ..., 345, 346, 347]])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [22]:
data.x[1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
data.edge_index

tensor([[236, 122,  24,  ...,   0,   0,   0],
        [186, 285, 346,  ..., 345, 346, 347]])

In [6]:
#獲得edge_features
# generate dummy edge features (5-dimensional vector of ones).
n_edge_channels = 5
edge_features = torch.ones([data.edge_index.shape[1], n_edge_channels])
edge_features

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [7]:
#建立好圖
data = Data(x=x, edge_index=edge_index, edge_attr=edge_features)
data_backup = Data(x=x, edge_index=edge_index, edge_attr=edge_features)
print(data)

Data(x=[348, 224], edge_index=[2, 5385], edge_attr=[5385, 5])


In [8]:
from torch_geometric.loader import NeighborSampler

In [9]:
train_loader = NeighborSampler(data.edge_index, sizes=[10, 10], batch_size=256,
                               shuffle=True, num_nodes=data.num_nodes)


In [10]:
#建立subgraph

In [11]:
#讀取子圖(circles)的節點
interested_nodes = []
with open(r'facebook\0.circles') as file:
    lines = file.readlines()
    a=0
    for line in lines:
        # 根據文件格式擷取circle節點
        b="circle"+str(a)
        features = line.strip(str(b)).split()
        features = [int(f) for f in features]
        features.append(0) #把egonode也加進circle裡面
#         print(features)
        interested_nodes.append(features)
        a=a+1
np.array(interested_nodes)
print(interested_nodes[0])

[71, 215, 54, 61, 298, 229, 81, 253, 193, 97, 264, 29, 132, 110, 163, 259, 183, 334, 245, 222, 0]


C:\Users\Yi-Chen\AppData\Local\Temp/ipykernel_3020/31169211.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(interested_nodes)


In [12]:
def subgraph(a): #a代表第a個子圖(circle)
    #建立egonode=0的subgraph
    b=0
    for b in range(1):#跑一次
        SUB_G=[]
        # 完整的圖數據中獲取感興趣節點和對應的邊索引
        subgraph_nodes = data.x[interested_nodes[a]]
        #把子圖節點索引轉成tensor
        interested_nodes_tensor = torch.tensor(interested_nodes[a])
        # Filter the edges based on the interested nodes
        mask = torch.logical_or(torch.isin(data.edge_index[0], interested_nodes_tensor),
                                torch.isin(data.edge_index[1], interested_nodes_tensor))
        #得到子圖的edge index
        SUB_E = data.edge_index[:, mask]
        #得到子圖的edge attr
        subgraph_edge_attr = data.edge_attr[torch.logical_or(
        torch.isin(data.edge_index[0], interested_nodes_tensor),
        torch.isin(data.edge_index[1], interested_nodes_tensor))]

        # 建構子圖 data
        SUB_G = Data(x=subgraph_nodes, edge_index=SUB_E, edge_attr=subgraph_edge_attr)
        print("第",a,"個subgraph data:")
        print(SUB_G)
        return SUB_G

In [13]:
#建立所有子圖
data1=subgraph(0)
data2=subgraph(1)
data3=subgraph(2)
data4=subgraph(3)
data5=subgraph(4)
data6=subgraph(5)
data7=subgraph(6)
data8=subgraph(7)
data9=subgraph(8)
data10=subgraph(9)
data11=subgraph(10)
data12=subgraph(11)
data13=subgraph(12)
data14=subgraph(13)
data15=subgraph(14)
data16=subgraph(15)
data17=subgraph(16)
data18=subgraph(17)
data19=subgraph(18)
data20=subgraph(19)
data21=subgraph(20)
data22=subgraph(21)
data23=subgraph(22)
data24=subgraph(23)

第 0 個subgraph data:
Data(x=[21, 224], edge_index=[2, 581], edge_attr=[581, 5])
第 1 個subgraph data:
Data(x=[2, 224], edge_index=[2, 357], edge_attr=[357, 5])
第 2 個subgraph data:
Data(x=[10, 224], edge_index=[2, 491], edge_attr=[491, 5])
第 3 個subgraph data:
Data(x=[4, 224], edge_index=[2, 377], edge_attr=[377, 5])
第 4 個subgraph data:
Data(x=[18, 224], edge_index=[2, 1007], edge_attr=[1007, 5])
第 5 個subgraph data:
Data(x=[2, 224], edge_index=[2, 379], edge_attr=[379, 5])
第 6 個subgraph data:
Data(x=[21, 224], edge_index=[2, 657], edge_attr=[657, 5])
第 7 個subgraph data:
Data(x=[3, 224], edge_index=[2, 373], edge_attr=[373, 5])
第 8 個subgraph data:
Data(x=[2, 224], edge_index=[2, 349], edge_attr=[349, 5])
第 9 個subgraph data:
Data(x=[11, 224], edge_index=[2, 443], edge_attr=[443, 5])
第 10 個subgraph data:
Data(x=[5, 224], edge_index=[2, 397], edge_attr=[397, 5])
第 11 個subgraph data:
Data(x=[31, 224], edge_index=[2, 1483], edge_attr=[1483, 5])
第 12 個subgraph data:
Data(x=[2, 224], edge_index=[2,

In [14]:
print(data1)

Data(x=[21, 224], edge_index=[2, 581], edge_attr=[581, 5])


In [15]:
#建立datasets

In [16]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['.\MyOwn.dataset']

    def download(self):
        pass

    def process(self):
        # Read data into huge `Data` list.
        data_list = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [17]:
dataset=MyOwnDataset('C:\\Users\\Yi-Chen\\python\\Final')

In [18]:
dataset = dataset.shuffle()
len(dataset)

10

In [19]:
#分割資料集
train_dataset = dataset[:7]
val_dataset = dataset[7:8]
test_dataset = dataset[8:]
len(train_dataset), len(val_dataset), len(test_dataset)

(7, 1, 2)

In [20]:
data=train_dataset[0]

In [10]:
from typing import Optional
from typing import Union, Tuple
from torch import Tensor
import torch.nn as nn
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import OptPairTensor, Adj, Size, OptTensor
from torch_geometric.utils import softmax
from torch_sparse import SparseTensor
import torch.nn.functional as F

In [11]:
class GraFrankConv(MessagePassing):
    """
    Modality-specific neighbor aggregation in GraFrank implemented by stacking message-passing layers that are
    parameterized by friendship attentions over individual node features and pairwise link features.
    """

    def __init__(self, in_channels: Union[int, Tuple[int, int]],
                 out_channels: int, normalize: bool = False,
                 bias: bool = True, **kwargs):  # yapf: disable
        kwargs.setdefault('aggr', 'add')
        super(GraFrankConv, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize
        self.negative_slope = 0.2
        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        self.self_linear = nn.Linear(in_channels[1], out_channels, bias=bias)
        self.message_linear = nn.Linear(in_channels[0], out_channels, bias=bias)

        self.attn = nn.Linear(out_channels, 1, bias=bias)
        self.attn_i = nn.Linear(out_channels, 1, bias=bias)

        self.lin_l = nn.Linear(out_channels, out_channels, bias=bias)
        self.lin_r = nn.Linear(out_channels, out_channels, bias=False)

        self.reset_parameters()
        self.dropout = 0

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj, edge_attr: OptTensor = None,
                size: Size = None) -> Tensor:
        if isinstance(x, Tensor):
            x: OptPairTensor = (x, x)

        x_l, x_r = x[0], x[1]
        self_emb = self.self_linear(x_r)
        alpha_i = self.attn_i(self_emb)
        out = self.propagate(edge_index, x=(x_l, x_r), alpha=alpha_i, edge_attr=edge_attr, size=size)
        out = self.lin_l(out) + self.lin_r(self_emb)  # dense layer.

        if self.normalize:
            out = F.normalize(out, p=2., dim=-1)

        return out

    def message(self, x_j: Tensor, alpha_i: Tensor, edge_attr: Tensor, index: Tensor, ptr: OptTensor,
                size_i: Optional[int]) -> Tensor:
        message = torch.cat([x_j, edge_attr], dim=-1)
        out = self.message_linear(message)
        alpha = self.attn(out) + alpha_i
        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, index, ptr, size_i)
        self._alpha = alpha
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        out = out * alpha
        return out

    def message_and_aggregate(self, adj_t: SparseTensor) -> Tensor:
        pass

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)


class CrossModalityAttention(nn.Module):
    """
    Cross-Modality Fusion in GraFrank implemented by an attention mechanism across the K modalities.
    """

    def __init__(self, hidden_channels):
        super(CrossModalityAttention, self).__init__()
        self.hidden_channels = hidden_channels
        self.multi_linear = nn.Linear(hidden_channels, hidden_channels, bias=True)
        self.multi_attn = nn.Sequential(self.multi_linear, nn.Tanh(), nn.Linear(hidden_channels, 1, bias=True))

    def forward(self, modality_x_list):
        """

        :param modality_x_list: list of modality-specific node embeddings.
        :return: final node embedding after fusion.
        """
        result = torch.cat([x.relu().unsqueeze(-2) for x in modality_x_list], -2)  # [...., K, hidden_channels]
        wts = torch.softmax(self.multi_attn(result).squeeze(-1), dim=-1)
        return torch.sum(wts.unsqueeze(-1) * self.multi_linear(result), dim=-2)

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.hidden_channels,
                                   self.hidden_channels)

In [12]:
class GraFrank(nn.Module):
    """
    GraFrank Model for Multi-Faceted Friend Ranking with multi-modal node features and pairwise link features.
    (a) Modality-specific neighbor aggregation: modality_convs
    (b) Cross-modality fusion layer: cross_modality_attention
    """

    def __init__(self, in_channels, hidden_channels, edge_channels, num_layers, input_dim_list):
        """

        :param in_channels: total cardinality of node features.
        :param hidden_channels: latent embedding dimensionality.
        :param edge_channels: number of link features.
        :param num_layers: number of message passing layers.
        :param input_dim_list: list containing the cardinality of node features per modality.
        """
        super(GraFrank, self).__init__()
        self.num_layers = num_layers
        self.modality_convs = nn.ModuleList()
        self.edge_channels = edge_channels
        # we assume that the input features are first partitioned and then concatenated across the K modalities.
        self.input_dim_list = input_dim_list

        for inp_dim in self.input_dim_list:
            modality_conv_list = nn.ModuleList()
            for i in range(num_layers):
                in_channels = in_channels if i == 0 else hidden_channels
                modality_conv_list.append(GraFrankConv((inp_dim + edge_channels, inp_dim), hidden_channels))

            self.modality_convs.append(modality_conv_list)

        self.cross_modality_attention = CrossModalityAttention(hidden_channels)

    def forward(self, x, adjs, edge_attrs):
        """ Compute node embeddings by recursive message passing, followed by cross-modality fusion.

        :param x: node features [B', in_channels] where B' is the number of nodes (and neighbors) in the mini-batch.
        :param adjs: list of sampled edge indices per layer (EdgeIndex format in PyTorch Geometric) in the mini-batch.
        :param edge_attrs: [E', edge_channels] where E' is the number of sampled edge indices per layer in the mini-batch.
        :return: node embeddings. [B, hidden_channels] where B is the number of target nodes in the mini-batch.
        """
        result = []
        for k, convs_k in enumerate(self.modality_convs):
            emb_k = None
            for i, ((edge_index, _, size), edge_attr) in enumerate(zip(adjs, edge_attrs)):
                x_target = x[:size[1]]  # Target nodes are always placed first.
                x_list = torch.split(x, split_size_or_sections=self.input_dim_list, dim=-1)  # modality partition
                x_target_list = torch.split(x_target, split_size_or_sections=self.input_dim_list, dim=-1)
                x_k, x_target_k = x_list[k], x_target_list[k]

                emb_k = convs_k[i]((x_k, x_target_k), edge_index, edge_attr=edge_attr)

                if i != self.num_layers - 1:
                    emb_k = emb_k.relu()
                    emb_k = F.dropout(emb_k, p=0.5, training=self.training)

            result.append(emb_k)
        return self.cross_modality_attention(result)

    def full_forward(self, x, edge_index, edge_attr):
        """ Auxiliary function to compute node embeddings for all nodes at once for small graphs.

        :param x: node features [N, in_channels] where N is the total number of nodes in the graph.
        :param edge_index: edge indices [2, E] where E is the total number of edges in the graph.
        :param edge_attr: link features [E, edge_channels] across all edges in the graph.
        :return: node embeddings. [N, hidden_channels] for all nodes in the graph.
        """
        x_list = torch.split(x, split_size_or_sections=self.input_dim_list, dim=-1)  # modality partition
        result = []
        for k, convs_k in enumerate(self.modality_convs):
            x_k = x_list[k]
            emb_k = None
            for i, conv in enumerate(convs_k):
                emb_k = conv(x_k, edge_index, edge_attr=edge_attr)

                if i != self.num_layers - 1:
                    emb_k = emb_k.relu()
                    emb_k = F.dropout(emb_k, p=0.5, training=self.training)

            result.append(emb_k)
#         print(result)
        return self.cross_modality_attention(result)

In [13]:
from torch_geometric.nn.conv import SAGEConv

In [14]:
class SAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers):
        super(SAGE, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(SAGEConv((in_channels, in_channels), hidden_channels))

    def forward(self, x, adjs, edge_attrs):
        for i, ((edge_index, _, size), edge_attr) in enumerate(zip(adjs, edge_attrs)):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

    def full_forward(self, x, edge_index, edge_attr):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

In [15]:
model_type = 'GraFrank'
# n_edge_channels = 5
if model_type == 'GraFrank':
    model = GraFrank(data.num_node_features, hidden_channels=64, edge_channels=n_edge_channels, num_layers=2,
                     input_dim_list=[224])  # input dim list assumes that the node features are first
    # partitioned and then concatenated across the K modalities.
else:
    model = SAGE(data.num_node_features, hidden_channels=64, num_layers=2)
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x = data.x

In [16]:
# loader = DataLoader(dataset, batch_size=256, shuffle=True)

In [17]:
class LinkPredictionModel(nn.Module):
    def __init__(self, num_users, embedding_dim):
        super(LinkPredictionModel, self).__init__()
        self.embedding = nn.Embedding(num_users, embedding_dim)
        self.fc = nn.Linear(embedding_dim, 1)

    def forward(self, user_indices):
        user_embeddings = self.embedding(user_indices)
        scores = self.fc(user_embeddings)
        return torch.sigmoid(scores)


In [18]:
#產生postiv&negativ edges
from torch_geometric.utils import train_test_split_edges

# Specify the ratio of edges to keep for training (e.g., 0.8 for 80%)
train_ratio = 0.8

# Split the edges into train and test sets
data_edge = train_test_split_edges(data_backup, val_ratio=0.8,test_ratio=0.1)
print(data_edge.val_pos_edge_index[1])


tensor([ 96, 185, 141,  ...,  62, 177, 180])


C:\Users\Yi-Chen\anaconda3\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [19]:
print(data_edge.val_neg_edge_index[1])

tensor([ 54, 197, 335,  ..., 286, 303, 186])


In [20]:
def train(loader):
    model.train()
    print(model)
    total_loss = 0
    it = 0
    num_users=348
    for batch_size, n_id, adjs in loader:
        it += 1
        edge_attrs = [data.edge_attr[e_id] for (edge_index, e_id, size) in adjs]
        adjs = [adj for adj in adjs]
        edge_attrs = [edge_attr for edge_attr in edge_attrs]

        optimizer.zero_grad()
        out = model(x[n_id], adjs, edge_attrs) #final user representations
        print(out)
#         embeds = out.tolist()
#         embeds = tuple(embeds)
#         print(embeds)

        # Sample a direct neighbor as a positive example
#         positive_indices = data_edge.val_pos_edge_index[1]  # Assuming the second dimension represents the target nodes
#         positive_indices = random.choice(positive_example_indices)
#         print(positive_indices[0])

        # Sample a random node as a negative example
#         all_nodes = torch.arange(data.num_nodes)  # Assuming 'data' has a property 'num_nodes' representing the total number of nodes
#         negative_indices = data_edge.val_neg_edge_index[1]

#         targets = torch.cat([torch.ones(len(positive_indices)), torch.zeros(len(negative_indices))])
#         print(targets)
#         loss = F.binary_cross_entropy(predictions.squeeze(), targets)

#         # Perform backpropagation and update model parameters
#         loss.backward()
#         # optimizer.step()  # Uncomment if using an optimizer

#         # Extract probabilities and threshold to determine predicted links
#         predicted_links = (predictions > 0.5).squeeze().tolist()

#         print("Predicted links:", predicted_links)
        
        out, pos_out, neg_out = out.split(out.size(0) // 3, dim=0)


        # binary skipgram loss can be replaced with margin-based pairwise ranking loss.
        pos_loss = F.logsigmoid((out * pos_out).sum(-1)).mean()
        neg_loss = F.logsigmoid(-(out * neg_out).sum(-1)).mean()
        loss = -pos_loss - neg_loss
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * out.size(0)

    return total_loss / data.num_nodes


In [21]:
train(train_loader)

GraFrank(
  (modality_convs): ModuleList(
    (0): ModuleList(
      (0): GraFrankConv((229, 224), 64)
      (1): GraFrankConv((229, 224), 64)
    )
  )
  (cross_modality_attention): CrossModalityAttention(64, 64)
)
tensor([[-0.0014,  0.0364,  0.0376,  ..., -0.0767,  0.1671,  0.0011],
        [-0.0221,  0.0423,  0.0561,  ..., -0.0959,  0.1294,  0.0467],
        [-0.0005,  0.0313,  0.0396,  ..., -0.0767,  0.1338, -0.0086],
        ...,
        [ 0.0136,  0.0611,  0.0784,  ..., -0.0711,  0.1433,  0.0302],
        [-0.0050,  0.0633,  0.0032,  ..., -0.0931,  0.1294,  0.0190],
        [-0.0138,  0.0587,  0.0085,  ..., -0.1033,  0.1658,  0.0208]],
       grad_fn=<SumBackward1>)


ValueError: too many values to unpack (expected 3)

In [34]:
def test():
    x, edge_index, edge_attr = data.x.to(device), data.edge_index.to(device), data.edge_attr.to(device)
    model.eval()
    out = model.full_forward(x, edge_index, edge_attr).cpu()
    return out


for epoch in range(1, 51):
    loss = train(train_loader)
    test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

GraFrank(
  (modality_convs): ModuleList(
    (0): ModuleList(
      (0): GraFrankConv((229, 224), 64)
      (1): GraFrankConv((229, 224), 64)
    )
  )
  (cross_modality_attention): CrossModalityAttention(64, 64)
)


IndexError: index 2146 is out of bounds for dimension 0 with size 357

In [31]:
# Example usage:
num_users = 100
embedding_dim = 64
# model = LinkPredictionModel(num_users, embedding_dim)

# Generate example data
user_indices = data.edge_index[1]  # User indices for prediction
positive_indices = data_edge.val_pos_edge_index  # Positive link indices
negative_indices = data_edge.val_neg_edge_index  # Negative link indices

# Compute predictions
predictions = model(user_indices)

# Compute loss (e.g., binary cross entropy)
targets = torch.cat([torch.ones(len(positive_indices)), torch.zeros(len(negative_indices))])
loss = F.binary_cross_entropy(predictions.squeeze(), targets)

# Perform backpropagation and update model parameters
loss.backward()
# optimizer.step()  # Uncomment if using an optimizer

# Extract probabilities and threshold to determine predicted links
predicted_links = (predictions > 0.5).squeeze().tolist()

print("Predicted links:", predicted_links)

TypeError: forward() missing 2 required positional arguments: 'adjs' and 'edge_attrs'

In [52]:
class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embed_size):
        super(LightGCN, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embed_size = embed_size

        self.user_embedding = nn.Embedding(num_users, embed_size)
        self.item_embedding = nn.Embedding(num_items, embed_size)

    def forward(self, user_indices, item_indices):
        user_embeds = self.user_embedding(user_indices)
        item_embeds = self.item_embedding(item_indices)
        return user_embeds, item_embeds

In [53]:
print(data_edge.val_neg_edge_index[1])

tensor([341, 316, 249,  ..., 327, 322, 143])


In [54]:
def train_model(data, embed_size, num_epochs, learning_rate):
    num_users, num_items, user_indices, item_indices = preprocess_data(data)
    
    model = LightGCN(num_users, num_items, embed_size)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        user_embeds, item_embeds = model(user_indices, item_indices)
        loss = criterion(user_embeds, item_embeds)
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    return model

In [57]:
import torch.optim as optim

def preprocess_data(data):
    num_users = data.num_nodes
    num_items = data.num_nodes
    user_indices = data_edge.val_pos_edge_index
    item_indices = data_edge.val_neg_edge_index
    
    return num_users, num_items, user_indices, item_indices


# 训练模型
embed_size = 64
num_epochs = 100
learning_rate = 0.001
model = train_model(data, embed_size, num_epochs, learning_rate)
print(model)

Epoch 1/100, Loss: 1.9780051708221436
Epoch 11/100, Loss: 1.9451525211334229
Epoch 21/100, Loss: 1.9128049612045288
Epoch 31/100, Loss: 1.881006121635437
Epoch 41/100, Loss: 1.8497785329818726
Epoch 51/100, Loss: 1.8191258907318115
Epoch 61/100, Loss: 1.7890429496765137
Epoch 71/100, Loss: 1.759519100189209
Epoch 81/100, Loss: 1.7305418252944946
Epoch 91/100, Loss: 1.7020972967147827
LightGCN(
  (user_embedding): Embedding(348, 64)
  (item_embedding): Embedding(348, 64)
)


In [67]:
loader = DataLoader(dataset, batch_size=512, shuffle=True)
for batch in loader:
    batch

In [ ]:
# 子圖怎麼合起來用
# Train/test怎麼做資料集的分割

In [44]:
# Create a list of Data objects
data_list = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10]


# combined_dataset = dataset.shuffle()
# len(combined_dataset)


# Create a DataLoader for the combined dataset
loader = DataLoader(data_list, batch_size=32, shuffle=True) #shuffle()方法確保資料集被隨機打亂
len(loader)

1

In [1]:
from torch_geometric.nn import GCNConv

conv = GCNConv(in_channels, out_channels)  # 创建图卷积层
output = conv(data.x, data.edge_index)  # 进行图卷积操作，输出结果

NameError: name 'in_channels' is not defined

In [4]:
# 從文件中讀取節點特徵向量
def read_node_features(file_path):
    with open(file_path) as file:
        lines = file.readlines()
        node_features = []
        a=1
        for line in lines:
            # 根據文件格式擷取節點特徵向量
            features = line.strip(str(a)).split()
            features = [float(f) for f in features]
            node_features.append(features)
            a=a+1
        return np.array(node_features)

# 讀取節點特徵向量數據
node_features = read_node_features(r'facebook\0.feat')

In [41]:
edgeo_index=subgraph_data.edge_index
print(edgeo_index.t())
subgraph_data.num_nodes

tensor([[307,  71],
        [334, 252],
        [ 27,  54],
        ...,
        [  0, 345],
        [  0, 346],
        [  0, 347]])


21